In [23]:
import pandas as pd
from pathlib import Path

file_path=Path("Resources/fire_data/mapdataall.csv")
df=pd.read_csv(file_path, encoding="utf-8")

In [24]:
df.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://incidentupdate.fire.ca.gov/incidents/2...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,https://incidentupdate.fire.ca.gov/incidents/2...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://incidentupdate.fire.ca.gov/incidents/2...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://incidentupdate.fire.ca.gov/incidents/2...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://incidentupdate.fire.ca.gov/incidents/2...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False


In [25]:
#gather list of original columns
#list(df.columns)

In [32]:
#create new data frame with select columns
fire_df = df[['incident_name',
 'incident_is_final',
 'incident_date_last_update',
 'incident_date_created',
 'incident_administrative_unit',
 'incident_county',
 'incident_acres_burned',
 'incident_cooperating_agencies',
 'incident_longitude',
 'incident_latitude',
 'incident_type',
]]

fire_df.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_county,incident_acres_burned,incident_cooperating_agencies,incident_longitude,incident_latitude,incident_type
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,Shasta,37.0,Shasta-Trinity National Forest,-122.309000,40.774000,NaN
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,San Diego,122.0,CAL FIRE San Diego Unit,1.000000,1.000000,Wildfire
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,Inyo,407.0,"CAL FIRE San Bernardino Unit, Inyo County Sher...",-118.016510,36.602575,NaN
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,San Bernardino,30.0,San Bernardino National Forest,-116.941311,34.288877,NaN
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,CAL FIRE Madera-Mariposa-Merced Unit,-119.635004,37.116295,NaN
